<a href="https://colab.research.google.com/github/kangwonlee/nmisp/blob/search-duffing-6dof/50_ode/60_Duffing_Oscillator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


In [ ]:
# This cell is for the Google Colaboratory
# https://stackoverflow.com/a/63519730
if 'google.colab' in str(get_ipython()):
  path_py = '/content/nmisp_py'

  import os
  if not os.path.exists(path_py):
    import subprocess
    subprocess.run(
        ('git', 'clone', 'https://github.com/kangwonlee/nmisp_py')
    )
  assert os.path.exists(path_py)

  import sys
  sys.path.insert(0, path_py)



In [ ]:
import os
import pathlib

import matplotlib.pyplot as plt
import matplotlib.gridspec as mg
import numpy as np



In [ ]:
import ode_solver



# Duffing Oscillator<br>더핑 진동자



The Duffing oscillator is a simple model of a mass-spring system with a nonlinear spring. The nonlinearity arises from the fact that the spring's restoring force is not proportional to its displacement. The Duffing oscillator is described by the following second-order differential equation:



$$
m\frac{d^2}{dt^2}x(t) + c\frac{d}{dt}x(t) + k x(t) + \alpha x^3 = F cos \omega t
$$


| symbol<br>기호 | unit<br>단위 | description<br>설명 |
|:-------------:|:-------------:|-------------|
|  $t$ | sec | time |
|  $x(t)$ | m | the displacement of the mass from its equilibrium position |
|  $\frac{d}{dt}x(t)$ | m/s | the velocity of the mass. |
|  $\frac{d^2}{dt^2}x(t)$ | m/s/s | the acceleration of the mass. |
|  $m$ | kg | the mass of the object attached to the spring. |
|  $c$ | N/(m/s) | the damping coefficient. |
|  $k$ | N/m | the linear stiffness coefficient of the spring. |
|  $\alpha$ | $N/{m^3}$ | the nonlinear stiffness coefficient of the spring. |
|  $F$ | N | the amplitude of the external driving force. |
|  $\omega$ | rad/sec | the angular frequency of the external driving force. |



## The Nonlinear Term

The key feature of the Duffing equation is the term $\alpha x^3$. This term introduces a cubic nonlinearity into the system.  When the displacement $x$ is small, this term is negligible, and the system behaves like a simple harmonic oscillator. However, as the displacement increases, the nonlinear term becomes significant, leading to deviations from simple harmonic motion.

## Physical Interpretation

The Duffing oscillator can model a variety of physical systems, including:

* Mechanical Systems: A mass-spring system where the spring's stiffness changes as it is stretched or compressed.
* Electrical Circuits: An RLC circuit with a nonlinear inductor or capacitor.
* Structural Systems: A beam or plate with large deflections, where the geometric nonlinearities become important.

## Chaotic Behavior

For certain combinations of parameters (m, c, k, alpha, F, omega), the Duffing oscillator can exhibit chaotic behavior. This means that the system's response is extremely sensitive to initial conditions, and its long-term behavior is unpredictable.

## Numerical Solution

The Duffing equation is a nonlinear differential equation, and in most cases, it does not have an analytical solution. Therefore, numerical methods, such as the Runge-Kutta methods, are essential for simulating the behavior of the Duffing oscillator.


In [ ]:
m_kg = 1.0
c_Npmps = 0.25
k_Npm = 1
alpha_Npm3 = 0.1
F_N = 0.5
omega_rps = 1.5



In [ ]:
def get_Duffing_oscillator_slope(
        m_kg:float, c_Npmps:float, k_Npm:float, alpha_Npm3:float,
        F_N:float, omega_rps:float):
    '''
    Return a closure calculating slope function
    기울기 함수를 계산하는 내포 함수를 반환
    '''

    neg_m_inv = (-1.0) / m_kg
    neg_c_m = c_Npmps * neg_m_inv
    neg_k_m = k_Npm * neg_m_inv
    neg_alpha_m = alpha_Npm3 * neg_m_inv
    F_m = F_N / m_kg

    def f(t:float, xv:np.ndarray,):
        '''
        Calculate slope vector of the Duffing Oscillator and a linear oscillator
        더핑 진동자와 선형 진동계의 기울기 벡터를 계산
        '''
        x, v, x2, v2 = xv
        dx_dt = v
        dv_dt = neg_k_m * x + neg_c_m * v + neg_alpha_m * x**3 + F_m * np.cos(omega_rps * t)
        dx2_dt = v2
        dv2_dt = neg_k_m * x2 + neg_c_m * v2 + F_m * np.cos(omega_rps * t)

        return np.array((dx_dt, dv_dt, dx2_dt, dv2_dt))

    return f



In [ ]:
def run_sim(
        slope, x_0:np.ndarray=np.array((0.0, 0.0, 0.0, 0.0)),
        t_start=0.0, t_end=120.0, delta_t_sec=1e-3,
    ):
    '''
    Simulate a one degree of freedom vibration system
    1자유도 진동계 시뮬레이션
    '''

    if os.getenv('CI', False):
        t_end = t_start + 1.0

    t_array = np.arange(t_start, t_end, delta_t_sec)

    t_out, x_out = ode_solver.rk4(slope, t_array, x_0)
    x_out = np.array(x_out)
    return t_out, x_out


def sim_1dof_vib(
        slope, x_0:np.ndarray=np.array((0.0, 0.0, 0.0, 0.0)),
        t_start=0.0, t_end=120.0, figsize=(14, 14),
        delta_t_sec=1e-3,
    ):

    t_out, x_out = run_sim(
        slope, x_0,
        t_start, t_end, delta_t_sec,
    )

    fig = plt.figure(figsize=figsize)
    gs = mg.GridSpec(2, 2, height_ratios=[1, 1], width_ratios=[1, 1])

    axs = (
        fig.add_subplot(gs[0, 0]),
        fig.add_subplot(gs[1, 0]),
        fig.add_subplot(gs[:, 1])
    )

    for k in range(2):
        axs[k].plot(t_out, x_out[:, k], label='Duffing')
        axs[k].plot(t_out, x_out[:, k+2], label='Linear')
        axs[k].grid(True)
        axs[k].set_xlabel('t(sec)')
    axs[0].set_ylabel('$x(t)$')
    axs[1].set_ylabel(r'$\frac{d}{dt}x(t)$')

    axs[2].plot(x_out[:, 0], x_out[:, 1], label='Duffing')
    axs[2].plot(x_out[:, 2], x_out[:, 3], label='Linear')
    axs[2].grid(True)
    axs[2].set_xlabel('$x(t)$')
    axs[2].set_ylabel(r'$\frac{d}{dt}x(t)$')

    return axs



In [ ]:
def explorer_duffing(F_N, omega_rps):
    Duffing_oscillator = get_Duffing_oscillator_slope(m_kg, c_Npmps, k_Npm, alpha_Npm3, F_N, omega_rps)
    ax = sim_1dof_vib(Duffing_oscillator,)
    return ax



* Weak forcing near the natural frequency.



In [ ]:
%time explorer_duffing(F_N=0.1, omega_rps=1)



* Stronger forcing, potentially leading to nonlinear effects.



In [ ]:
%time explorer_duffing(F_N=0.5, omega_rps=1.5)



* Exploring different resonance regions.



In [ ]:
%time explorer_duffing(F_N=1.0, omega_rps=0.5)



In [ ]:
%time explorer_duffing(F_N=1.0, omega_rps=1.0)



In [ ]:
%time explorer_duffing(F_N=1.0, omega_rps=2.0)



* Sweeping omega to look for period-doubling bifurcations and chaos.



In [ ]:
%time explorer_duffing(F_N=1.5, omega_rps=0.8)



In [ ]:
%time explorer_duffing(F_N=1.5, omega_rps=1.2)



* Strong forcing showing nonlinear force effects



In [ ]:
%time explorer_duffing(F_N=2.5, omega_rps=0.48)



In [ ]:
import scipy.optimize as so




In [ ]:
%%writefile parallel_cost.py
import ast
import dataclasses
import logging
import pathlib
from typing import ClassVar, List, Tuple


import numpy as np
import numpy.linalg as nl
import scipy.integrate as si


FORMAT = '%(asctime)s - %(levelname)s - %(message)s'
logging.basicConfig(encoding='utf-8', level=logging.INFO, format=FORMAT)


@dataclasses.dataclass
class Duffing:
    '''
    Implement a closure calculating slope function
    기울기 함수를 계산하는 내포 함수를 구현
    '''
    m_kg:float=1.0
    c_Npmps:float=0.25
    k_Npm:float=1
    alpha_Npm3:float=0.1
    F_N:float=0.5
    omega_rps:float=1.5

    def __post_init__(self):
        self.neg_m_inv = (-1.0) / self.m_kg
        self.neg_c_m = self.c_Npmps * self.neg_m_inv
        self.neg_k_m = self.k_Npm * self.neg_m_inv
        self.neg_alpha_m = self.alpha_Npm3 * self.neg_m_inv
        self.F_m = self.F_N / self.m_kg
        self.freq_rps = (self.k_Npm / self.m_kg) ** 0.5
        self.period_sec = 1.0 / self.freq_rps

    def __call__(self, *argv, **kwarg):
        return self.slope(*argv, **kwarg)

    def slope(self, t:float, xv:np.ndarray,):
        '''
        Calculate slope vector of the Duffing Oscillator and a linear oscillator
        더핑 진동자와 선형 진동계의 기울기 벡터를 계산
        '''
        x, v, x2, v2 = xv
        dx_dt = v
        dv_dt = self.neg_k_m * x + self.neg_c_m * v + self.neg_alpha_m * x**3 + self.F_m * np.cos(self.omega_rps * t)
        dx2_dt = v2
        dv2_dt = self.neg_k_m * x2 + self.neg_c_m * v2 + self.F_m * np.cos(self.omega_rps * t)

        return np.array((dx_dt, dv_dt, dx2_dt, dv2_dt))


@dataclasses.dataclass
class ParallelCost:
    '''
    calculate cost function given parameters
    주어진 매개변수로 비용함수를 계산
    '''
    p:pathlib.Path=pathlib.Path('min.txt')
    delta_t_sec:float=5e-6
    t_start_sec:float=0.0
    t_end_sec:float=120.0
    x0_0:np.ndarray=None
    x0_1:np.ndarray=None
    lam:float = 0.5 # exponential weight

    def __post_init__(self):
        self.t_eval = np.arange(
            self.t_start_sec,
            self.t_end_sec+self.delta_t_sec*0.5,
            self.delta_t_sec
        )

        # to make sure t_eval within t_span
        self.t_eval = self.t_eval[self.t_eval<=self.t_end_sec]

        if self.x0_0 is None:
            self.x0_0 = np.array((0.0, 0.0, 0.0, 0.0))

        if self.x0_1 is None:
            self.x0_1 = np.array((0.01, 0.01, 0.01, 0.01))

    def append(self, new_line):
        with self.p.open('at') as f:
            f.write((new_line + '\n'))

    def cost(self, x):
        if any(np.isnan(x)):
            logging.info(f'x has nan : {x}')
            return 1e5
        m_kg, c_Npmps, k_Npm, alpha_Npm3, F_N, omega_rps = x

        Duffing_oscillator = Duffing(m_kg, c_Npmps, k_Npm, alpha_Npm3, F_N, omega_rps)
        sol0 = si.solve_ivp(
            fun=Duffing_oscillator,
            t_span=(self.t_start_sec, self.t_end_sec),
            y0=self.x0_0,
            t_eval=self.t_eval,
            method='BDF',
        )
        t0, x0 = sol0.t, sol0.y

        if not sol0.success:
            logging.warning(f'sol0.success = {sol0.success}')
            logging.warning(f'sol0.message = {sol0.message}')
            logging.warning(
                f'param = {m_kg:g}, {c_Npmps:g}, {k_Npm:g}, '
                f'{alpha_Npm3:g}, {F_N:g}, {omega_rps:g}'
            )
            return 1e5

        if np.abs(x0([:, 0]).max() > 100.0:
            logging.warning(f'x0[:, 0].max() = {x0[:, 0].max()}')
            logging.warning(f'x0[:, 1].max() = {x0[:, 1].max()}')
            logging.warning(
                f'param = {m_kg:g}, {c_Npmps:g}, {k_Npm:g}, '
                f'{alpha_Npm3:g}, {F_N:g}, {omega_rps:g}'
            )
            return 1e5

        if not isinstance(sol0.y, np.ndarray):
            logging.warning(f'type(sol0.y) = {type(sol0.y)}')
            logging.warning(f'sol0.success = {sol0.success}')
            logging.warning(f'sol0.y = {sol0.y}')
            logging.warning(
                f'param = {m_kg:g}, {c_Npmps:g}, {k_Npm:g}, '
                f'{alpha_Npm3:g}, {F_N:g}, {omega_rps:g}'
            )
            return 1e5

        assert len(self.t_eval) == sol0.y.shape[1], (
            f'sol0.success = {sol0.success}\n'
            f'len(self.t_eval) = {len(self.t_eval)} sol0.y.shape = {sol0.y.shape}\n'
            f'm_kg={m_kg:g}, c_Npmps={c_Npmps:g}, k_Npm={k_Npm:g}, '
            f'alpha_Npm3={alpha_Npm3:g}, F_N={F_N:g}, omega_rps={omega_rps:g}'
        )

        sol1 = si.solve_ivp(
            fun=Duffing_oscillator,
            t_span=(self.t_start_sec, self.t_end_sec),
            y0=self.x0_1,
            t_eval=self.t_eval,
            method='BDF',
        )
        t1, x1 = sol1.t, sol1.y

        if not sol1.success:
            logging.warning(f'sol1.success = {sol1.success}')
            logging.warning(f'sol0.success = {sol0.success}')
            logging.warning(
                f'param = {m_kg:g}, {c_Npmps:g}, {k_Npm:g}, '
                f'{alpha_Npm3:g}, {F_N:g}, {omega_rps:g}'
            )
            return 1e5

        new_cost = - self.l_exp(t0, x0[:2, :], t1, x1[:2, :])

        del x0
        del t0
        del x1
        del t1
        del sol0
        del sol1

        return (new_cost)

    def l_exp(self, t0, x0, t1, x1):
        '''
        Calculate the Lyapunov exponent
        The larger, the more chaotic
        '''
        assert len(t0) == x0.shape[1], (
            f'length different len(t0) = {len(t0)}, x0.shape = {x0.shape}'
        )
        assert 2 == x0.shape[0], f'x0.shape = {x0.shape}'
        assert x0.shape == x1.shape, f'x0.shape = {x0.shape} x1.shape = {x1.shape}'
        assert np.allclose(t0, t1), 'time different'

        n = len(x0)
        separation = nl.norm(x0 - x1, axis=0)

        if 0.0 == separation.min():

            t0 = t0[separation>0.0]
            separation = separation[separation>0.0]

        log_separations = np.log(separation)

        coeffs = np.polyfit(t0, log_separations, 1)
        return coeffs[0]  # Slope of the line is the Lyapunov exponent estimate

    def read_min(self) -> List[Tuple[float]]:
        txt = self.p.read_text()
        t = ast.literal_eval('['+txt+']')
        t.sort()
        return t

    def append_min(self, txt:str):
        with self.p.open('a') as f:
            f.write(txt)

    def de_callback(self, intermediate_result, convergence):
        '''
        optimizer will call once per iteration
        '''

        # TODO : consider storing in memory
        t = self.read_min()

        # calculate cost
        c = self.cost(intermediate_result)

        if t[0][0] > c:
            new_line = f'''({c}, {','.join(map(str, intermediate_result))}),\n'''
            logging.info(new_line)
            # append the new low
            logging.info('new low')
            self.append_min(new_line)



In [ ]:
import parallel_cost as pc



In [ ]:
import ast
import logging
import pathlib


FORMAT = '%(asctime)s - %(levelname)s - %(message)s'
logging.basicConfig(encoding='utf-8', level=logging.INFO, format=FORMAT)


def read_min(path='min.txt'):
    p_min = pathlib.Path(path)
    if p_min.exists():
        logging.info(f'reading {p_min}')
        txt = p_min.read_text()
        t = ast.literal_eval('['+txt+']')
        t.sort()
        m_array = np.array(t)
        #m_init = m_array[m_array[:, 0] < 0, 1:]
        m_init = m_array[:, 1:]
        logging.info(f'm_init.shape = {m_init.shape}')
    else:
        m_init = None
        logging.info(f'm_init = {m_init}')

    return m_init



In [ ]:
%%time

bounds = [
    (0.9, 1.1), # m [kg]
    (0.1, 1.0), # c [N/(m/s)]
    (0.1, 10.0), # k [N/m]
    (-1.0, 1.0), # alpha [N/m3]
    (0.1, 10.0), # F [N]
    (0.1, 5.0), # omega [rad/sec]
]

max_iter = 1 if os.getenv('CI', False) else 1000

pcost = pc.ParallelCost()
logging.info('start')
result = so.differential_evolution(
    pcost.cost, bounds, popsize=1000,
    workers=(-1),
    updating='deferred',
    init=read_min(),
    callback=pcost.de_callback,
    maxiter=max_iter,
)
logging.info('end')

result



In [ ]:
m_kg, c_Npmps, k_Npm, alpha_Npm3, F_N, omega_rps = result.x

Duffing_oscillator = get_Duffing_oscillator_slope(m_kg, c_Npmps, k_Npm, alpha_Npm3, F_N, omega_rps)
ax = sim_1dof_vib(Duffing_oscillator,)



In [ ]:
ax = sim_1dof_vib(Duffing_oscillator, x_0=np.array((0.01, 0.01, 0.01, 0.01)))

